<div class="alert" style="background-color:##fff; padding:0px 10px; border-radius:5px;">
    <div align="center">
        <img src="https://www.mad.tf.fau.de/files/2019/04/cropped-logo_mad.png" width=400>
        <hr>
    </div>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;">
    <h2 style='margin:10px 5px'> 
        Content
    </h2>
</div>

- [1. Import Packages](#1)
- [2. Download Podcast by RSS Feed Link](#2)
    - [2.1 Download Podcast as mp3](#2.1)
    - [2.2 Generate metadata.csv separately](#2.2)
- [3. Moving Podcasts from External Drive to local Dir](#3)
- [4. Create and Move the Metadata for all Podcasts](#4)

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;">
    <h2 style='margin:10px 5px'> 
        <a name='1' style="color:white; text-decoration:none;">
            1. Import Packages
        </a>
    </h2>
</div>

In [1]:
import wget
import feedparser
import glob
import sys
import os
import requests
import datetime
import shutil
import pandas as pd
import json

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from scripts.podcasts import get_input, validate_input, get_feed, download_files

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;">
    <h2 style='margin:10px 5px'> 
        <a name='2' style="color:white; text-decoration:none;">
            2. Download podcast by RSS Feed Link
        </a>
    </h2>
</div>

The following code downloads all podcast episodes of a certain podcast into the local directory, which can be defined and re-set by `local_target_dir`.

In [2]:
# RSS Feed to download and local location to download files too
# RSS_Target = <https link to the RSS feed> "https://securityinfive.libsyn.com/rss"
# local_target_dir = <local directory 'C:\foldername\target' format> "C:\Dev\PodcastArchive"

local_target_dir = os.getcwd()

In [3]:
RSS_Target = "https://feeds.blubrry.com/feeds/beckershealthcarepodcast.xml"

RSS_Feed = get_feed(RSS_Target)
RSS_Feed_Items = RSS_Feed[0]
RSS_Feed_Count = RSS_Feed[1]

Getting feed - https://feeds.blubrry.com/feeds/beckershealthcarepodcast.xml please wait...
Feed loaded.


<a name='2.1'></a>
### 2.1 Download Podcast as MP3

In [4]:
RSS_Feed = get_feed(RSS_Target)
RSS_Feed_Items = RSS_Feed[0]
RSS_Feed_Count = RSS_Feed[1]

"""
    is_valid = 0 -> Download all episodes
    is_valid > 0 -> Download that amount of episodes
"""
is_valid = 0

if datetime.date.today().year < 2022:
    
    while is_valid == 0:
        How_Many = get_input(RSS_Feed_Count)
        is_valid = validate_input(How_Many, RSS_Feed_Count)
    else:
        if int(How_Many) == 0:
            File_Count = "All files to - "
        elif int(How_Many) == 1:
            File_Count = str(How_Many) + " file to - "
        else:
            File_Count = str(How_Many) + " files to - "
        print("Downloading " + File_Count + local_target_dir)
        Download_Files(local_target_dir, RSS_Feed_Items, How_Many)
else:
    print("No Download - Podcast used only until the end of 2021")

Getting feed - https://thebigunlock.com/feed/podcast please wait...
Feed loaded.
No Download - Podcast used only until the end of 2021


<a name='2.2'></a>
### 2.2 Generate metadata.csv separately

> In case metadata is missing for the episodes, it is necessary to download them manually again.

In [4]:
RSS_PODCASTS = ['15 Minutes With The Doctor', 'Beckers Healthcare Podcast',
       'Conversations on Health Care', 'Data Book',
       'Digital Health Today', 'Faces of Digital Health',
       'GeekWire Health Tech', 'Harlow on Healthcare',
       'Health Care Rounds', 'HealthChangers', 'Healthcare IT Today',
       'Healthcare Rap', 'Healthcare Strategies',
       'Healthcare Triage Podcast',
       'Healthcare Weekly - At the Forefront of Healthcare Innovation',
       'Hospitals In Focus', 'How I Transformed This', 'Medtech Talk',
       'Outcomes Rocket', 'Perspectives on Health and Tech',
       'PopHealth Week',
       'Solving Healthcare with Dr. Kwadwo Kyeremanteng',
       'Talking HealthTech', 'The #HCBiz Show!', 'The Big Unlock',
       'The Change Healthcare Podcast', 'The Incrementalist',
       'The Pulse by Wharton Digital Health', 'The Tate Chronicles',
       'This Just In']

In [6]:
with open("../../data/rss.json", 'r') as j:
     rss_feed = json.loads(j.read())

In [7]:
metadata_csv = pd.DataFrame(columns={"podcast", "episode", "published"})

In [8]:
parsing_podcast = False

if parsing_podcast:

    for i in range(len(RSS_PODCASTS)):

        feed_link = rss_feed[RSS_PODCASTS[i]]

        # get the rss data for each podcast
        RSS_Target = feed_link
        RSS_Feed = get_feed(RSS_Target)
        RSS_Feed_Items = RSS_Feed[0]

        for episode in RSS_Feed_Items["entries"]:
            published = episode["published_parsed"]
            date = str(published.tm_year) + "-" + str(published.tm_mon) + "-" + str(published.tm_mday)
            metadata_csv = metadata_csv.append({"podcast": RSS_PODCASTS[i], 
                                                "episode": episode["title"], 
                                                "published": str(date)}, ignore_index=True)

        print(f"Successfully parsed {RSS_PODCASTS[i]} and appended {len(RSS_Feed_Items['entries'])} entries.")
        
    metadata_csv.to_csv("new_metadata.csv")

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;">
    <h2 style='margin:10px 5px'> 
        <a name='3' style="color:white; text-decoration:none;">
            3. Moving podcasts from external drive to local directory
        </a>
    </h2>
</div>

After transcribing all podcast episodes with the APIs Vosk (and DeepSpeech) onto the external drive due to the high volume, the text files will be moved into the local directory and the output folder will be deleted.

In [8]:
podcast_path = "/mnt/d/master-thesis-long-do/podcasts"
thesis_podcast_path = "/mnt/c/Users/timlo/PycharmProjects/master-thesis-tim-loehr/podcasts"

podcast_names = os.listdir(podcast_path)

for podcast in podcast_names:
    dir_path = os.path.join(podcast_path, podcast)
    output_path = os.path.join(dir_path, "output")
    thesis_podcast = os.path.join(thesis_podcast_path, podcast)
    
    if os.path.isdir(thesis_podcast):
        
        for text in os.listdir(output_path):
            text_path = os.path.join(output_path, text)
            target_path = os.path.join(thesis_podcast, text)
            shutil.copyfile(text_path, target_path)
        
    else:
        os.mkdir(thesis_podcast)
        pass
    
episode_count = 0

for podcast in podcast_names:
    pc_path = os.path.join(podcast_path, podcast)
    podcasts = os.listdir(pc_path)
    podcasts_mp3 = podcasts[podcasts[:-4] == "'.mp3'"]
    episode_count += len(podcasts_mp3)
    
print(f"Number Episodes: {episode_count}")

Number Episodes: 2011


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;">
    <h2 style='margin:10px 5px'> 
        <a name='4' style="color:white; text-decoration:none;">
            4. Create and move the metadata for all podcasts
        </a>
    </h2>
</div>

The metadata for all podcasts are stored in one central `csv` file, so it is easier managable.

In [10]:
metadata_path = "../../data/metadata/metadata.csv"
metadata_orig_path = "/mnt/d/master-thesis-long-do/podcasts/_metadata"

if not os.path.isfile(metadata_path):

    df = pd.DataFrame()

    for meta in os.listdir(metadata_orig_path):
        path = os.path.join(metadata_orig_path, meta)
        shutil.copyfile(path, "../../data/metadata/outputs/" + meta)
        
        podcast_df = pd.read_csv(path)
        df = df.append(podcast_df)

    df.to_csv("../../data/metadata/metadata.csv", index=False)
    print("Successfully created metadata.csv and moves to ", metadata_path)
    
else:
    print("metadata.csv already exists")

Successfully created metadata.csv and moves to  ../../data/metadata/metadata.csv
